# Mistral OCR avec Annotations

Ce notebook utilise l'API Mistral OCR pour extraire le texte et les images des documents PDF avec des annotations structurées.

## Fonctionnalités :
- Extraction du texte avec conservation de la structure
- Extraction et annotation des images, graphiques, tableaux
- Export des images extraites dans un dossier
- Génération d'un fichier Markdown propre

In [30]:
import os
import base64
import json
from pathlib import Path
from dotenv import load_dotenv
from mistralai import Mistral
from mistralai.models import OCRResponse
from pydantic import BaseModel, Field
from enum import Enum
from IPython.display import Markdown, display

# Charger les variables d'environnement
load_dotenv()

# Initialiser le client Mistral
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

print("✅ Client Mistral initialisé avec succès")

✅ Client Mistral initialisé avec succès


## Définition des modèles Pydantic pour les Annotations

Nous définissons les schémas pour :
- **BBox Annotation** : Type d'image (graphique, tableau, texte, image) et description
- **Document Annotation** : Langue, résumé, auteurs

In [31]:
# Définition des types d'images pour BBox Annotation
class ImageType(str, Enum):
    GRAPH = "graph"
    CHART = "chart"
    TABLE = "table"
    SCHEMA = "schema"
    DIAGRAM = "diagram"
    FIGURE = "figure"
    IMAGE = "image"
    TEXT = "text"
    SIGNATURE = "signature"
    LOGO = "logo"

# Modèle pour l'annotation des BBox (images extraites)
class ImageAnnotation(BaseModel):
    image_type: ImageType = Field(..., description="Le type de l'image détectée (graph, chart, table, schema, diagram, figure, image, text, signature, logo)")
    title: str = Field(..., description="Titre ou légende de l'image si disponible")
    short_description: str = Field(..., description="Description courte de l'image en français")
    detailed_description: str = Field(..., description="Description détaillée du contenu de l'image en français")
    data_extracted: str = Field(default="", description="Données extraites si c'est un tableau ou graphique (format texte)")

# Modèle pour l'annotation du document complet
class DocumentAnnotation(BaseModel):
    language: str = Field(..., description="Langue du document en format ISO 639-1 (ex: 'fr', 'en')")
    title: str = Field(..., description="Titre principal du document")
    document_type: str = Field(..., description="Type de document (rapport, procès-verbal, contrat, etc.)")
    summary: str = Field(..., description="Résumé complet du document en français")
    key_points: list[str] = Field(..., description="Points clés du document")
    authors: list[str] = Field(default=[], description="Liste des auteurs ou signataires du document")
    date: str = Field(default="", description="Date du document si mentionnée")
    organizations: list[str] = Field(default=[], description="Organisations ou entités mentionnées")

print("✅ Modèles d'annotation définis")

✅ Modèles d'annotation définis


## Fonctions utilitaires

Fonctions pour :
- Encoder les fichiers PDF/images en base64
- Traiter la réponse OCR
- Sauvegarder les images extraites
- Générer le Markdown final

In [32]:
def encode_file_to_base64(file_path: str) -> str:
    """
    Encode un fichier (PDF ou image) en base64.
    
    Args:
        file_path: Chemin vers le fichier
        
    Returns:
        Chaîne base64 encodée
    """
    try:
        with open(file_path, "rb") as file:
            return base64.b64encode(file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"❌ Erreur: Le fichier {file_path} n'a pas été trouvé.")
        return None
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None


def get_mime_type(file_path: str) -> str:
    """
    Détermine le type MIME en fonction de l'extension du fichier.
    """
    ext = Path(file_path).suffix.lower()
    mime_types = {
        '.pdf': 'application/pdf',
        '.png': 'image/png',
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.gif': 'image/gif',
        '.webp': 'image/webp',
        '.avif': 'image/avif',
        '.bmp': 'image/bmp',
        '.tiff': 'image/tiff',
    }
    return mime_types.get(ext, 'application/octet-stream')


def save_base64_image(base64_string: str, output_path: str) -> bool:
    """
    Sauvegarde une image base64 vers un fichier.
    
    Args:
        base64_string: Image encodée en base64 (peut inclure le préfixe data:...)
        output_path: Chemin de sortie pour l'image
        
    Returns:
        True si succès, False sinon
    """
    try:
        # Retirer le préfixe data:image/...;base64, si présent
        if base64_string.startswith('data:'):
            base64_string = base64_string.split(',', 1)[1]
        
        image_data = base64.b64decode(base64_string)
        
        with open(output_path, 'wb') as f:
            f.write(image_data)
        return True
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde de l'image: {e}")
        return False


def create_output_directory(base_name: str) -> Path:
    """
    Crée le dossier de sortie pour les résultats OCR.
    
    Args:
        base_name: Nom de base du document
        
    Returns:
        Chemin du dossier créé
    """
    output_dir = Path("output") / base_name
    images_dir = output_dir / "images"
    
    output_dir.mkdir(parents=True, exist_ok=True)
    images_dir.mkdir(parents=True, exist_ok=True)
    
    return output_dir


print("✅ Fonctions utilitaires définies")

✅ Fonctions utilitaires définies


In [33]:
import yaml

def parse_document_annotation(annotation_str: str) -> dict:
    """
    Parse l'annotation du document (JSON string) en dictionnaire.
    """
    if not annotation_str:
        return {}
    try:
        return json.loads(annotation_str)
    except:
        return {"raw": annotation_str}


def parse_image_annotation(annotation_str: str) -> dict:
    """
    Parse l'annotation d'image (JSON string) en dictionnaire.
    """
    if not annotation_str:
        return {}
    try:
        return json.loads(annotation_str)
    except:
        return {"description": annotation_str}


def generate_frontmatter(
    source_file: str,
    document_annotation: str,
    images_info: list[dict],
    total_pages: int
) -> str:
    """
    Génère le frontmatter YAML pour Astro.
    
    Args:
        source_file: Nom du fichier source
        document_annotation: Annotation du document (JSON string)
        images_info: Liste des infos sur les images
        total_pages: Nombre total de pages
        
    Returns:
        Frontmatter YAML formaté
    """
    # Parser l'annotation du document
    doc_data = parse_document_annotation(document_annotation)
    
    # Construire le frontmatter
    frontmatter = {
        "source_file": source_file,
        "total_pages": total_pages,
        "total_images": len(images_info),
    }
    
    # Ajouter les données du document si disponibles
    if doc_data:
        if "title" in doc_data:
            frontmatter["title"] = doc_data.get("title", "")
        if "language" in doc_data:
            frontmatter["language"] = doc_data.get("language", "")
        if "document_type" in doc_data:
            frontmatter["document_type"] = doc_data.get("document_type", "")
        if "summary" in doc_data:
            frontmatter["summary"] = doc_data.get("summary", "")
        if "key_points" in doc_data:
            frontmatter["key_points"] = doc_data.get("key_points", [])
        if "authors" in doc_data:
            frontmatter["authors"] = doc_data.get("authors", [])
        if "date" in doc_data:
            frontmatter["date"] = doc_data.get("date", "")
        if "organizations" in doc_data:
            frontmatter["organizations"] = doc_data.get("organizations", [])
    
    # Ajouter les annotations des images
    if images_info:
        images_data = []
        for img in images_info:
            img_entry = {
                "id": img.get("id", ""),
                "filename": img.get("filename", ""),
                "page": img.get("page", 0),
            }
            
            # Parser et ajouter l'annotation de l'image
            if "annotation" in img and img["annotation"]:
                img_annotation = parse_image_annotation(img["annotation"])
                if img_annotation:
                    img_entry["image_type"] = img_annotation.get("image_type", "")
                    img_entry["title"] = img_annotation.get("title", "")
                    img_entry["description"] = img_annotation.get("short_description", "") or img_annotation.get("description", "")
                    img_entry["detailed_description"] = img_annotation.get("detailed_description", "")
                    if "data_extracted" in img_annotation and img_annotation["data_extracted"]:
                        img_entry["data_extracted"] = img_annotation.get("data_extracted", "")
            
            images_data.append(img_entry)
        
        frontmatter["images"] = images_data
    
    # Générer le YAML
    yaml_content = yaml.dump(frontmatter, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    return f"---\n{yaml_content}---\n"


def process_ocr_response(
    ocr_response: OCRResponse, 
    output_dir: Path,
    include_annotations: bool = True,
    source_filename: str = ""
) -> tuple[str, list[dict]]:
    """
    Traite la réponse OCR et génère le markdown avec frontmatter Astro.
    
    Args:
        ocr_response: Réponse de l'API OCR
        output_dir: Dossier de sortie
        include_annotations: Inclure les annotations dans le markdown
        source_filename: Nom du fichier source
        
    Returns:
        Tuple (markdown_content, images_info)
    """
    images_dir = output_dir / "images"
    markdowns = []
    images_info = []
    
    # D'abord, collecter toutes les images pour le frontmatter
    for page_idx, page in enumerate(ocr_response.pages):
        for img_idx, img in enumerate(page.images):
            img_id = img.id
            
            if hasattr(img, 'image_base64') and img.image_base64:
                base64_str = img.image_base64
                
                # Déterminer le format
                if base64_str.startswith('data:image/png'):
                    ext = '.png'
                elif base64_str.startswith('data:image/jpeg') or base64_str.startswith('data:image/jpg'):
                    ext = '.jpg'
                else:
                    ext = '.png'
                
                img_filename = f"page{page_idx + 1}_img{img_idx + 1}{ext}"
                img_path = images_dir / img_filename
                
                if save_base64_image(base64_str, str(img_path)):
                    print(f"  ✅ Image sauvegardée: {img_filename}")
                    
                    img_info = {
                        "id": img_id,
                        "filename": img_filename,
                        "page": page_idx + 1,
                        "position": {
                            "top_left_x": getattr(img, 'top_left_x', None),
                            "top_left_y": getattr(img, 'top_left_y', None),
                            "bottom_right_x": getattr(img, 'bottom_right_x', None),
                            "bottom_right_y": getattr(img, 'bottom_right_y', None),
                        }
                    }
                    
                    if hasattr(img, 'image_annotation') and img.image_annotation:
                        img_info["annotation"] = img.image_annotation
                    
                    images_info.append(img_info)
    
    # Générer le frontmatter YAML pour Astro
    document_annotation = getattr(ocr_response, 'document_annotation', None)
    frontmatter = generate_frontmatter(
        source_file=source_filename,
        document_annotation=document_annotation,
        images_info=images_info,
        total_pages=len(ocr_response.pages)
    )
    markdowns.append(frontmatter)
    
    # Traiter chaque page pour le contenu markdown
    for page_idx, page in enumerate(ocr_response.pages):
        image_data = {}
        
        # Préparer les données des images pour cette page
        for img_info in images_info:
            if img_info["page"] == page_idx + 1:
                img_id = img_info["id"]
                relative_path = f"images/{img_info['filename']}"
                
                # Créer l'alt text à partir de l'annotation
                alt_text = img_id
                if "annotation" in img_info and img_info["annotation"]:
                    img_annotation = parse_image_annotation(img_info["annotation"])
                    if img_annotation:
                        # Construire un alt text descriptif
                        parts = []
                        if img_annotation.get("image_type"):
                            parts.append(f"[{img_annotation['image_type']}]")
                        if img_annotation.get("title"):
                            parts.append(img_annotation["title"])
                        elif img_annotation.get("short_description"):
                            parts.append(img_annotation["short_description"])
                        elif img_annotation.get("description"):
                            parts.append(img_annotation["description"])
                        
                        if parts:
                            alt_text = " - ".join(parts)
                
                image_data[img_id] = {
                    "path": relative_path,
                    "alt": alt_text
                }
        
        # Remplacer les références d'images dans le markdown
        page_markdown = page.markdown
        for img_id, data in image_data.items():
            old_ref = f"![{img_id}]({img_id})"
            # Utiliser l'annotation comme alt text
            new_ref = f"![{data['alt']}]({data['path']})"
            page_markdown = page_markdown.replace(old_ref, new_ref)
        
        markdowns.append(page_markdown)
    
    return "\n\n".join(markdowns), images_info


print("✅ Fonction de traitement OCR avec frontmatter Astro définie")

✅ Fonction de traitement OCR avec frontmatter Astro définie


## Fonction principale d'OCR avec Annotations

Cette fonction traite un document PDF ou une image et extrait :
- Le texte avec structure préservée
- Les images/schémas/tableaux avec leurs annotations
- Les métadonnées du document

In [ ]:
from mistralai.extra import response_format_from_pydantic_model

def run_ocr_with_annotations(
    file_path: str,
    use_bbox_annotation: bool = True,
    use_document_annotation: bool = True,
    max_pages: int = 8
) -> dict:
    """
    Exécute l'OCR avec annotations sur un document.
    Génère un markdown avec frontmatter YAML compatible Astro.
    
    Args:
        file_path: Chemin vers le fichier PDF ou image
        use_bbox_annotation: Activer l'annotation des images/graphiques
        use_document_annotation: Activer l'annotation du document entier
        max_pages: Nombre maximum de pages pour document_annotation (limite API: 8)
        
    Returns:
        Dictionnaire avec les résultats
    """
    file_path = Path(file_path)
    base_name = file_path.stem
    
    print(f"🔄 Traitement du fichier: {file_path.name}")
    print("-" * 50)
    
    # Encoder le fichier en base64
    base64_content = encode_file_to_base64(str(file_path))
    if not base64_content:
        return None
    
    print("✅ Fichier encodé en base64")
    
    # Déterminer le type de document
    mime_type = get_mime_type(str(file_path))
    is_pdf = mime_type == 'application/pdf'
    
    # Préparer la configuration du document
    if is_pdf:
        document_config = {
            "type": "document_url",
            "document_url": f"data:{mime_type};base64,{base64_content}"
        }
    else:
        document_config = {
            "type": "image_url",
            "image_url": f"data:{mime_type};base64,{base64_content}"
        }
    
    print(f"📄 Type de document: {mime_type}")
    
    # Préparer les paramètres de l'appel OCR
    ocr_params = {
        "model": "mistral-ocr-latest",
        "document": document_config,
        "include_image_base64": True
    }
    
    # Ajouter les formats d'annotation si demandés
    if use_bbox_annotation:
        ocr_params["bbox_annotation_format"] = response_format_from_pydantic_model(ImageAnnotation)
        print("✅ Annotation BBox activée")
    
    if use_document_annotation:
        ocr_params["document_annotation_format"] = response_format_from_pydantic_model(DocumentAnnotation)
        ocr_params["pages"] = list(range(max_pages))  # Limite de 8 pages pour document_annotation
        print(f"✅ Annotation Document activée (max {max_pages} pages)")
    
    print("\n🔄 Appel de l'API Mistral OCR...")
    
    # Appel à l'API OCR
    try:
        ocr_response = client.ocr.process(**ocr_params)
        print("✅ OCR terminé avec succès!")
    except Exception as e:
        print(f"❌ Erreur lors de l'OCR: {e}")
        return None
    
    # Créer le dossier de sortie
    output_dir = create_output_directory(base_name)
    print(f"\n📁 Dossier de sortie créé: {output_dir}")
    
    # Traiter la réponse et générer le markdown avec frontmatter Astro
    print("\n🔄 Traitement des résultats...")
    markdown_content, images_info = process_ocr_response(
        ocr_response, 
        output_dir,
        include_annotations=(use_bbox_annotation or use_document_annotation),
        source_filename=file_path.name
    )
    
    # Sauvegarder le fichier markdown
    markdown_path = output_dir / f"{base_name}.md"
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    print(f"✅ Markdown avec frontmatter Astro sauvegardé: {markdown_path}")
    
    # Sauvegarder les métadonnées JSON (backup)
    metadata = {
        "source_file": str(file_path),
        "total_pages": len(ocr_response.pages),
        "total_images": len(images_info),
        "document_annotation": ocr_response.document_annotation if hasattr(ocr_response, 'document_annotation') else None,
        "images": images_info
    }
    
    metadata_path = output_dir / f"{base_name}_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    print(f"✅ Métadonnées sauvegardées: {metadata_path}")
    
    print("\n" + "=" * 50)
    print(f"📊 Résumé:")
    print(f"   - Pages traitées: {len(ocr_response.pages)}")
    print(f"   - Images extraites: {len(images_info)}")
    print(f"   - Dossier de sortie: {output_dir}")
    print("=" * 50)
    
    return {
        "output_dir": output_dir,
        "markdown_path": markdown_path,
        "metadata_path": metadata_path,
        "ocr_response": ocr_response,
        "markdown_content": markdown_content,
        "images_info": images_info,
        "metadata": metadata
    }


print("✅ Fonction principale d'OCR définie")

✅ Fonction principale d'OCR définie


## Traitement d'un document PDF

Exemple avec un des fichiers PDF du dossier `ressources/`

In [35]:
# Lister les fichiers disponibles dans le dossier ressources
ressources_dir = Path("ressources")
pdf_files = list(ressources_dir.glob("*.pdf"))

print("📂 Fichiers PDF disponibles dans 'ressources/':")
for i, pdf in enumerate(pdf_files, 1):
    print(f"   {i}. {pdf.name}")

📂 Fichiers PDF disponibles dans 'ressources/':
   1. 23248.pdf
   2. 5-qst-1975-03-01.pdf
   3. cadastraux_paris_09_54.pdf
   4. CAT_C_SR.229-FR.pdf
   5. circulaire-lir-n-1023-du-9-avril-1985.pdf
   6. commission_bruxelles_96.pdf
   7. compte-rendu-50-2003.pdf
   8. exportIr.pdf
   9. i1985_1986_0096_02.pdf
   10. main.pdf
   11. pv_1995-09-29.pdf
   12. s19951031_2329_2383.pdf


In [36]:
# Sélectionner le fichier à traiter (vous pouvez changer l'index ou le nom du fichier)
selected_pdf = pdf_files[0] if pdf_files else None

if selected_pdf:
    print(f"📄 Fichier sélectionné: {selected_pdf.name}")
    
    # Exécuter l'OCR avec annotations
    result = run_ocr_with_annotations(
        file_path=str(selected_pdf),
        use_bbox_annotation=True,
        use_document_annotation=True,
        max_pages=8
    )
else:
    print("❌ Aucun fichier PDF trouvé dans le dossier ressources/")

📄 Fichier sélectionné: 23248.pdf
🔄 Traitement du fichier: 23248.pdf
--------------------------------------------------
✅ Fichier encodé en base64
📄 Type de document: application/pdf
✅ Annotation BBox activée
✅ Annotation Document activée (max 8 pages)

🔄 Appel de l'API Mistral OCR...


✅ OCR terminé avec succès!

📁 Dossier de sortie créé: output\23248

🔄 Traitement des résultats...
  ✅ Image sauvegardée: page1_img1.jpg
  ✅ Image sauvegardée: page5_img1.jpg
✅ Markdown avec frontmatter Astro sauvegardé: output\23248\23248.md
✅ Métadonnées sauvegardées: output\23248\23248_metadata.json

📊 Résumé:
   - Pages traitées: 8
   - Images extraites: 2
   - Dossier de sortie: output\23248


## Affichage du résultat

Visualisation du markdown généré et des informations extraites

In [37]:
# Afficher le markdown généré (si un résultat existe)
if result:
    print("📄 Aperçu du Markdown généré:")
    print("=" * 50)
    display(Markdown(result["markdown_content"][:5000] + "\n\n...[Tronqué pour l'affichage]" if len(result["markdown_content"]) > 5000 else result["markdown_content"]))

📄 Aperçu du Markdown généré:


---
source_file: 23248.pdf
total_pages: 8
total_images: 2
title: Rapport d'Activité de l'ORSTOM
language: fr
document_type: rapport
summary: Le rapport d'activité de l'ORSTOM (Institut Français de Recherche Scientifique
  pour le Développement en Coopération) pour l'année 1985 décrit les efforts de renouvellement
  et de modernisation de l'institut. Il met en lumière les progrès réalisés dans divers
  départements de recherche, notamment en climatologie, en développement, et en coopération
  internationale. Le rapport souligne également les défis et les succès dans la mise
  en œuvre de nouveaux programmes et la modernisation des infrastructures. Il aborde
  également les efforts de coopération avec divers pays et organisations, ainsi que
  les résultats des recherches menées dans différentes régions du monde.
key_points:
- Efforts de renouvellement et de modernisation de l'ORSTOM en 1985.
- Progrès réalisés dans divers départements de recherche.
- Coopération internationale et nouveaux programmes.
- Modernisation des infrastructures et défis rencontrés.
- Résultats des recherches dans différentes régions du monde.
authors:
- Pierre LAVAU
- Alain RUELLAN
date: '1985'
organizations:
- ORSTOM
- Institut Français de Recherche Scientifique pour le Développement en Coopération
images:
- id: img-0.jpeg
  filename: page1_img1.jpg
  page: 1
  image_type: image
  title: Image avec des cercles
  description: Image montrant deux cercles gris sur un fond blanc
  detailed_description: L'image présente deux cercles gris superposés sur un fond
    blanc. Les cercles sont de taille similaire et sont placés de manière à se chevaucher
    partiellement. L'image semble être une partie d'une image plus grande, car elle
    est coupée sur les bords.
- id: img-1.jpeg
  filename: page5_img1.jpg
  page: 5
  image_type: image
  title: Image de paysage flou
  description: Une image en noir et blanc représentant un paysage flou.
  detailed_description: L'image montre un paysage en noir et blanc avec des zones
    de lumière et d'ombre. Les détails sont flous, rendant difficile l'identification
    des éléments spécifiques du paysage. La composition semble inclure des zones de
    végétation et peut-être des formations rocheuses, mais la faible résolution et
    le flou rendent toute identification précise impossible.
---


# RAPPORT D'ACTIVITÉ 

![[image] - Image avec des cercles](images/page1_img1.jpg)

4
Une Recherche Fondamentale pour l'Action :
Connaissances Climatiques et Développement

11
Les Départements,
Opérateurs de la Recherche

12
Département A :
Climat, Tectonique, Surface

19
Département B :
Evaluation et Contrôle des Repercussions
Induites par les Politiques de Développement

25
Département C :
Necessite d'une Démarche Intégrée

33
Département D :
Confirmation des Options
et Nouvelles Orientations

38
Département E :
Les Conditions de la Sécurité Alimentaire

43
Département F :
Energies, Eau, Matières Premières

47
Département G :
Pour l'Amélioration de la Santé,
Recherches Fomfamentales,
Nouveaux Outils Méthodologiques,
Stratégies de Prévention et de Contrôle

55
Département H :
Quelques Questions Ciels
Relatives an Développement

60
Annexe :
Grands Programmes et Départements

63
DIVA :
Des Outils et des Ouvertures
pour la Recherche

79
Services Scientifiques et Techniques

85
Services des Relations Extérieures :
Une Coopération Internationale en Extension

100
Les Moyens

101
Moyens en Personnels

106
Moyens Financiers

L'année 1985, ainsi que les premiers mois de 1986, peuvent être considérés comme la phase terminale des efforts de renouvellement, d'expansion et de modernisation de l'Institut entrepris en 1982.

Deux des obstacles majeurs au bon fonctionnement de l'ORSTOM ont enfin été levés en octobre 1985 avec l'installation du siége dans ses nouveaux locaux de la rue La Fayette et l'intervention du décret portant nouveau statut des personnels qui assure, pour l'essentiel, l'homogénéité de carrière avec les statuts des autres EPST.

D'autre part, l'obtention des crédits nécessaires dans les budgets de 1985 (et 1986) a permis de mettre en chantier la construction des futurs laboratoires de Montpellier (achèvement prévu en septembre 1987) et de moderniser les équipements scientifiques du Centre de Bondy.

Parallèlement a été poursuivi le redéploiement géographique et thématique entrepris avec constance depuis plusieurs années en vue de mieux équilibrer nos activités et de nouer ou renouer des rapports de coopération avec des partenaires susceptibles de diversifier davantage nos thèmes de recherche et points d'études comparatives.

Avec le Maroc au Maghreb, puis plus récemment la Guinée en Afrique occidentale et Madagascar dans l'Océan Indien ont été conclus des accords ouvrant la voie à une reprise de coopérations souhaitées de part et d'autre. En matière de recherche océanologique, le pôle Caraïbes a commencé à s'organiser, à partir de la Martinique, en association ave l'IFREMER et l'Université Antilles-Guyane.

Du

...[Tronqué pour l'affichage]

In [38]:
# Afficher les informations sur les images extraites
if result and result["images_info"]:
    print("🖼️ Images extraites:")
    print("=" * 50)
    for img in result["images_info"]:
        print(f"\n📷 {img['filename']} (Page {img['page']})")
        if "annotation" in img and img["annotation"]:
            print(f"   Annotation: {img['annotation'][:200]}..." if len(str(img['annotation'])) > 200 else f"   Annotation: {img['annotation']}")
else:
    print("ℹ️ Aucune image extraite du document")

🖼️ Images extraites:

📷 page1_img1.jpg (Page 1)
   Annotation: {
  "image_type": "image",
  "title": "Image avec des cercles",
  "short_description": "Image montrant deux cercles gris sur un fond blanc",
  "detailed_description": "L'image présente deux cercles gr...

📷 page5_img1.jpg (Page 5)
   Annotation: {
  "image_type": "image",
  "title": "Image de paysage flou",
  "short_description": "Une image en noir et blanc représentant un paysage flou.",
  "detailed_description": "L'image montre un paysage e...


## Traitement de tous les PDFs du dossier ressources

Fonction pour traiter tous les documents en lot

In [39]:
import time

def process_all_documents(
    input_dir: str = "ressources",
    file_pattern: str = "*.pdf",
    use_bbox_annotation: bool = True,
    use_document_annotation: bool = True,
    sleep_seconds: float = 2.0
) -> list[dict]:
    """
    Traite tous les documents d'un dossier avec pause entre chaque appel API.
    Génère des fichiers markdown avec frontmatter Astro.
    
    Args:
        input_dir: Dossier contenant les documents
        file_pattern: Pattern glob pour filtrer les fichiers
        use_bbox_annotation: Activer l'annotation des images
        use_document_annotation: Activer l'annotation du document
        sleep_seconds: Temps d'attente entre chaque document (pour free tier)
        
    Returns:
        Liste des résultats pour chaque document
    """
    input_path = Path(input_dir)
    files = list(input_path.glob(file_pattern))
    
    print(f"📂 {len(files)} fichiers trouvés dans '{input_dir}' avec le pattern '{file_pattern}'")
    print(f"⏱️ Pause de {sleep_seconds}s entre chaque document (free tier)")
    print("=" * 60)
    
    results = []
    
    for i, file_path in enumerate(files, 1):
        print(f"\n[{i}/{len(files)}] Traitement de: {file_path.name}")
        print("-" * 60)
        
        try:
            result = run_ocr_with_annotations(
                file_path=str(file_path),
                use_bbox_annotation=use_bbox_annotation,
                use_document_annotation=use_document_annotation
            )
            
            if result:
                results.append({
                    "file": file_path.name,
                    "status": "success",
                    "output_dir": str(result["output_dir"]),
                    "images_count": len(result["images_info"])
                })
            else:
                results.append({
                    "file": file_path.name,
                    "status": "failed",
                    "error": "OCR failed"
                })
        except Exception as e:
            results.append({
                "file": file_path.name,
                "status": "error",
                "error": str(e)
            })
            print(f"❌ Erreur: {e}")
        
        # Pause entre chaque document pour respecter les limites du free tier
        if i < len(files):
            print(f"\n⏳ Pause de {sleep_seconds}s avant le prochain document...")
            time.sleep(sleep_seconds)
    
    # Résumé final
    print("\n" + "=" * 60)
    print("📊 RÉSUMÉ DU TRAITEMENT EN LOT")
    print("=" * 60)
    
    success_count = sum(1 for r in results if r["status"] == "success")
    failed_count = len(results) - success_count
    
    print(f"✅ Réussis: {success_count}/{len(results)}")
    print(f"❌ Échecs: {failed_count}/{len(results)}")
    
    if success_count > 0:
        print(f"\n📁 Les fichiers ont été sauvegardés dans le dossier 'output/'")
        print("   Chaque sous-dossier contient:")
        print("   - Un fichier .md avec frontmatter YAML (compatible Astro)")
        print("   - Un fichier _metadata.json avec les données brutes")
        print("   - Un dossier images/ avec les images extraites")
    
    return results


print("✅ Fonction de traitement en lot définie (avec pause pour free tier)")

✅ Fonction de traitement en lot définie (avec pause pour free tier)


In [40]:
# Traiter TOUS les fichiers PDF du dossier ressources
# Avec pause de 2 secondes entre chaque document pour le free tier

batch_results = process_all_documents(
    input_dir="ressources",
    file_pattern="*.pdf",
    use_bbox_annotation=True,
    use_document_annotation=True,
    sleep_seconds=2.0  # Pause de 2 secondes pour free tier
)

📂 12 fichiers trouvés dans 'ressources' avec le pattern '*.pdf'
⏱️ Pause de 2.0s entre chaque document (free tier)

[1/12] Traitement de: 23248.pdf
------------------------------------------------------------
🔄 Traitement du fichier: 23248.pdf
--------------------------------------------------
✅ Fichier encodé en base64
📄 Type de document: application/pdf
✅ Annotation BBox activée
✅ Annotation Document activée (max 8 pages)

🔄 Appel de l'API Mistral OCR...
✅ OCR terminé avec succès!

📁 Dossier de sortie créé: output\23248

🔄 Traitement des résultats...
  ✅ Image sauvegardée: page1_img1.jpg
  ✅ Image sauvegardée: page5_img1.jpg
✅ Markdown avec frontmatter Astro sauvegardé: output\23248\23248.md
✅ Métadonnées sauvegardées: output\23248\23248_metadata.json

📊 Résumé:
   - Pages traitées: 8
   - Images extraites: 2
   - Dossier de sortie: output\23248

⏳ Pause de 2.0s avant le prochain document...

[2/12] Traitement de: 5-qst-1975-03-01.pdf
-------------------------------------------------

## Traitement d'une image unique

Exemple pour traiter une seule image (PNG, JPEG, etc.)

In [41]:
def run_ocr_on_image(
    image_path: str,
    use_bbox_annotation: bool = True
) -> dict:
    """
    Exécute l'OCR sur une image unique.
    
    Args:
        image_path: Chemin vers l'image
        use_bbox_annotation: Activer l'annotation
        
    Returns:
        Dictionnaire avec les résultats
    """
    image_path = Path(image_path)
    base_name = image_path.stem
    
    print(f"🖼️ Traitement de l'image: {image_path.name}")
    print("-" * 50)
    
    # Encoder l'image en base64
    base64_content = encode_file_to_base64(str(image_path))
    if not base64_content:
        return None
    
    print("✅ Image encodée en base64")
    
    # Déterminer le type MIME
    mime_type = get_mime_type(str(image_path))
    
    # Préparer l'appel OCR
    ocr_params = {
        "model": "mistral-ocr-latest",
        "document": {
            "type": "image_url",
            "image_url": f"data:{mime_type};base64,{base64_content}"
        },
        "include_image_base64": True
    }
    
    if use_bbox_annotation:
        ocr_params["bbox_annotation_format"] = response_format_from_pydantic_model(ImageAnnotation)
    
    print("🔄 Appel de l'API Mistral OCR...")
    
    try:
        ocr_response = client.ocr.process(**ocr_params)
        print("✅ OCR terminé!")
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None
    
    # Créer le dossier de sortie
    output_dir = create_output_directory(base_name)
    
    # Traiter la réponse
    markdown_content, images_info = process_ocr_response(
        ocr_response, 
        output_dir,
        include_annotations=use_bbox_annotation
    )
    
    # Sauvegarder
    markdown_path = output_dir / f"{base_name}.md"
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    print(f"✅ Résultat sauvegardé dans: {output_dir}")
    
    return {
        "output_dir": output_dir,
        "markdown_path": markdown_path,
        "markdown_content": markdown_content,
        "images_info": images_info
    }


print("✅ Fonction OCR pour images définie")

✅ Fonction OCR pour images définie


In [42]:
# Exemple d'utilisation pour une image
# image_result = run_ocr_on_image("chemin/vers/image.png")

## Structure de sortie pour Astro

Après le traitement, vous obtenez un dossier avec la structure suivante :

```
output/
└── nom_du_document/
    ├── nom_du_document.md          # Fichier Markdown avec frontmatter YAML
    ├── nom_du_document_metadata.json  # Métadonnées JSON (backup)
    └── images/
        ├── page1_img1.png          # Images extraites
        ├── page1_img2.png
        └── ...
```

### Format du frontmatter YAML (compatible Astro)

```yaml
---
source_file: document.pdf
total_pages: 5
total_images: 3
title: "Titre du document"
language: fr
document_type: "procès-verbal"
summary: "Résumé du document..."
key_points:
  - Point clé 1
  - Point clé 2
authors:
  - Auteur 1
date: "1995-09-29"
organizations:
  - Organisation 1
images:
  - id: img_001
    filename: page1_img1.png
    page: 1
    image_type: table
    title: "Titre de l'image"
    description: "Description courte"
    detailed_description: "Description détaillée"
---
```

### Format des images dans le markdown

Les images utilisent l'annotation comme texte alternatif :
```markdown
![{image_type} - {description}](images/page1_img1.png)
```

## Mode Multi-Pages : Un Markdown par page

Ce mode génère un fichier markdown séparé pour chaque page du document.
Structure de sortie dans `output_2/` :

```
output_2/
└── nom_du_document/
    ├── page_1.md
    ├── page_2.md
    ├── ...
    └── images/
        ├── page1_img1.png
        └── ...
```

In [43]:
def generate_page_frontmatter(
    source_file: str,
    page_number: int,
    total_pages: int,
    document_annotation: str,
    page_images_info: list[dict]
) -> str:
    """
    Génère le frontmatter YAML pour une page individuelle (Astro).
    
    Args:
        source_file: Nom du fichier source
        page_number: Numéro de la page (1-indexed)
        total_pages: Nombre total de pages
        document_annotation: Annotation du document complet
        page_images_info: Liste des images de cette page
        
    Returns:
        Frontmatter YAML formaté
    """
    doc_data = parse_document_annotation(document_annotation)
    
    frontmatter = {
        "source_file": source_file,
        "page_number": page_number,
        "total_pages": total_pages,
        "total_images": len(page_images_info),
    }
    
    # Ajouter les données du document si disponibles
    if doc_data:
        if "title" in doc_data:
            frontmatter["document_title"] = doc_data.get("title", "")
        if "language" in doc_data:
            frontmatter["language"] = doc_data.get("language", "")
        if "document_type" in doc_data:
            frontmatter["document_type"] = doc_data.get("document_type", "")
        # Le résumé n'est inclus que sur la première page
        if page_number == 1:
            if "summary" in doc_data:
                frontmatter["summary"] = doc_data.get("summary", "")
            if "key_points" in doc_data:
                frontmatter["key_points"] = doc_data.get("key_points", [])
            if "authors" in doc_data:
                frontmatter["authors"] = doc_data.get("authors", [])
            if "date" in doc_data:
                frontmatter["date"] = doc_data.get("date", "")
            if "organizations" in doc_data:
                frontmatter["organizations"] = doc_data.get("organizations", [])
    
    # Ajouter les annotations des images de cette page
    if page_images_info:
        images_data = []
        for img in page_images_info:
            img_entry = {
                "id": img.get("id", ""),
                "filename": img.get("filename", ""),
            }
            
            if "annotation" in img and img["annotation"]:
                img_annotation = parse_image_annotation(img["annotation"])
                if img_annotation:
                    img_entry["image_type"] = img_annotation.get("image_type", "")
                    img_entry["title"] = img_annotation.get("title", "")
                    img_entry["description"] = img_annotation.get("short_description", "") or img_annotation.get("description", "")
                    img_entry["detailed_description"] = img_annotation.get("detailed_description", "")
                    if "data_extracted" in img_annotation and img_annotation["data_extracted"]:
                        img_entry["data_extracted"] = img_annotation.get("data_extracted", "")
            
            images_data.append(img_entry)
        
        frontmatter["images"] = images_data
    
    yaml_content = yaml.dump(frontmatter, allow_unicode=True, default_flow_style=False, sort_keys=False)
    return f"---\n{yaml_content}---\n"


def create_output_directory_v2(base_name: str) -> Path:
    """
    Crée le dossier de sortie pour les résultats OCR multi-pages.
    
    Args:
        base_name: Nom de base du document
        
    Returns:
        Chemin du dossier créé
    """
    output_dir = Path("output_2") / base_name
    images_dir = output_dir / "images"
    
    output_dir.mkdir(parents=True, exist_ok=True)
    images_dir.mkdir(parents=True, exist_ok=True)
    
    return output_dir


def process_ocr_response_per_page(
    ocr_response: OCRResponse, 
    output_dir: Path,
    source_filename: str = ""
) -> tuple[list[Path], list[dict]]:
    """
    Traite la réponse OCR et génère un markdown par page avec frontmatter Astro.
    
    Args:
        ocr_response: Réponse de l'API OCR
        output_dir: Dossier de sortie
        source_filename: Nom du fichier source
        
    Returns:
        Tuple (liste des chemins markdown, images_info)
    """
    images_dir = output_dir / "images"
    markdown_paths = []
    all_images_info = []
    
    document_annotation = getattr(ocr_response, 'document_annotation', None)
    total_pages = len(ocr_response.pages)
    
    # D'abord, collecter et sauvegarder toutes les images
    for page_idx, page in enumerate(ocr_response.pages):
        for img_idx, img in enumerate(page.images):
            img_id = img.id
            
            if hasattr(img, 'image_base64') and img.image_base64:
                base64_str = img.image_base64
                
                if base64_str.startswith('data:image/png'):
                    ext = '.png'
                elif base64_str.startswith('data:image/jpeg') or base64_str.startswith('data:image/jpg'):
                    ext = '.jpg'
                else:
                    ext = '.png'
                
                img_filename = f"page{page_idx + 1}_img{img_idx + 1}{ext}"
                img_path = images_dir / img_filename
                
                if save_base64_image(base64_str, str(img_path)):
                    print(f"  ✅ Image sauvegardée: {img_filename}")
                    
                    img_info = {
                        "id": img_id,
                        "filename": img_filename,
                        "page": page_idx + 1,
                        "position": {
                            "top_left_x": getattr(img, 'top_left_x', None),
                            "top_left_y": getattr(img, 'top_left_y', None),
                            "bottom_right_x": getattr(img, 'bottom_right_x', None),
                            "bottom_right_y": getattr(img, 'bottom_right_y', None),
                        }
                    }
                    
                    if hasattr(img, 'image_annotation') and img.image_annotation:
                        img_info["annotation"] = img.image_annotation
                    
                    all_images_info.append(img_info)
    
    # Générer un fichier markdown par page
    for page_idx, page in enumerate(ocr_response.pages):
        page_number = page_idx + 1
        
        # Filtrer les images de cette page
        page_images = [img for img in all_images_info if img["page"] == page_number]
        
        # Générer le frontmatter pour cette page
        frontmatter = generate_page_frontmatter(
            source_file=source_filename,
            page_number=page_number,
            total_pages=total_pages,
            document_annotation=document_annotation,
            page_images_info=page_images
        )
        
        # Préparer les données des images pour le remplacement
        image_data = {}
        for img_info in page_images:
            img_id = img_info["id"]
            relative_path = f"images/{img_info['filename']}"
            
            alt_text = img_id
            if "annotation" in img_info and img_info["annotation"]:
                img_annotation = parse_image_annotation(img_info["annotation"])
                if img_annotation:
                    parts = []
                    if img_annotation.get("image_type"):
                        parts.append(f"[{img_annotation['image_type']}]")
                    if img_annotation.get("title"):
                        parts.append(img_annotation["title"])
                    elif img_annotation.get("short_description"):
                        parts.append(img_annotation["short_description"])
                    elif img_annotation.get("description"):
                        parts.append(img_annotation["description"])
                    
                    if parts:
                        alt_text = " - ".join(parts)
            
            image_data[img_id] = {
                "path": relative_path,
                "alt": alt_text
            }
        
        # Remplacer les références d'images dans le markdown
        page_markdown = page.markdown
        for img_id, data in image_data.items():
            old_ref = f"![{img_id}]({img_id})"
            new_ref = f"![{data['alt']}]({data['path']})"
            page_markdown = page_markdown.replace(old_ref, new_ref)
        
        # Combiner frontmatter et contenu
        full_content = frontmatter + "\n" + page_markdown
        
        # Sauvegarder le fichier markdown de la page
        page_filename = f"page_{page_number}.md"
        page_path = output_dir / page_filename
        
        with open(page_path, 'w', encoding='utf-8') as f:
            f.write(full_content)
        
        markdown_paths.append(page_path)
        print(f"  ✅ Page {page_number}/{total_pages} sauvegardée: {page_filename}")
    
    return markdown_paths, all_images_info


print("✅ Fonctions pour le mode multi-pages définies")

✅ Fonctions pour le mode multi-pages définies


In [44]:
def run_ocr_with_annotations_per_page(
    file_path: str,
    use_bbox_annotation: bool = True,
    use_document_annotation: bool = True,
    max_pages: int = 8
) -> dict:
    """
    Exécute l'OCR avec annotations et génère un markdown par page.
    Sortie dans output_2/
    
    Args:
        file_path: Chemin vers le fichier PDF ou image
        use_bbox_annotation: Activer l'annotation des images/graphiques
        use_document_annotation: Activer l'annotation du document entier
        max_pages: Nombre maximum de pages pour document_annotation (limite API: 8)
        
    Returns:
        Dictionnaire avec les résultats
    """
    file_path = Path(file_path)
    base_name = file_path.stem
    
    print(f"🔄 Traitement du fichier (mode multi-pages): {file_path.name}")
    print("-" * 50)
    
    # Encoder le fichier en base64
    base64_content = encode_file_to_base64(str(file_path))
    if not base64_content:
        return None
    
    print("✅ Fichier encodé en base64")
    
    # Déterminer le type de document
    mime_type = get_mime_type(str(file_path))
    is_pdf = mime_type == 'application/pdf'
    
    # Préparer la configuration du document
    if is_pdf:
        document_config = {
            "type": "document_url",
            "document_url": f"data:{mime_type};base64,{base64_content}"
        }
    else:
        document_config = {
            "type": "image_url",
            "image_url": f"data:{mime_type};base64,{base64_content}"
        }
    
    print(f"📄 Type de document: {mime_type}")
    
    # Préparer les paramètres de l'appel OCR
    ocr_params = {
        "model": "mistral-ocr-latest",
        "document": document_config,
        "include_image_base64": True
    }
    
    # Ajouter les formats d'annotation si demandés
    if use_bbox_annotation:
        ocr_params["bbox_annotation_format"] = response_format_from_pydantic_model(ImageAnnotation)
        print("✅ Annotation BBox activée")
    
    if use_document_annotation:
        ocr_params["document_annotation_format"] = response_format_from_pydantic_model(DocumentAnnotation)
        ocr_params["pages"] = list(range(max_pages))
        print(f"✅ Annotation Document activée (max {max_pages} pages)")
    
    print("\n🔄 Appel de l'API Mistral OCR...")
    
    # Appel à l'API OCR
    try:
        ocr_response = client.ocr.process(**ocr_params)
        print("✅ OCR terminé avec succès!")
    except Exception as e:
        print(f"❌ Erreur lors de l'OCR: {e}")
        return None
    
    # Créer le dossier de sortie (output_2)
    output_dir = create_output_directory_v2(base_name)
    print(f"\n📁 Dossier de sortie créé: {output_dir}")
    
    # Traiter la réponse et générer un markdown par page
    print("\n🔄 Génération des fichiers markdown par page...")
    markdown_paths, images_info = process_ocr_response_per_page(
        ocr_response, 
        output_dir,
        source_filename=file_path.name
    )
    
    # Sauvegarder les métadonnées JSON globales
    metadata = {
        "source_file": str(file_path),
        "total_pages": len(ocr_response.pages),
        "total_images": len(images_info),
        "document_annotation": ocr_response.document_annotation if hasattr(ocr_response, 'document_annotation') else None,
        "markdown_files": [str(p.name) for p in markdown_paths],
        "images": images_info
    }
    
    metadata_path = output_dir / f"{base_name}_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    print(f"✅ Métadonnées sauvegardées: {metadata_path}")
    
    print("\n" + "=" * 50)
    print(f"📊 Résumé:")
    print(f"   - Pages traitées: {len(ocr_response.pages)}")
    print(f"   - Fichiers markdown créés: {len(markdown_paths)}")
    print(f"   - Images extraites: {len(images_info)}")
    print(f"   - Dossier de sortie: {output_dir}")
    print("=" * 50)
    
    return {
        "output_dir": output_dir,
        "markdown_paths": markdown_paths,
        "metadata_path": metadata_path,
        "ocr_response": ocr_response,
        "images_info": images_info,
        "metadata": metadata
    }


print("✅ Fonction OCR multi-pages définie")

✅ Fonction OCR multi-pages définie


In [45]:
def process_all_documents_per_page(
    input_dir: str = "ressources",
    file_pattern: str = "*.pdf",
    use_bbox_annotation: bool = True,
    use_document_annotation: bool = True,
    sleep_seconds: float = 2.0
) -> list[dict]:
    """
    Traite tous les documents et génère un markdown par page.
    Sortie dans output_2/
    
    Args:
        input_dir: Dossier contenant les documents
        file_pattern: Pattern glob pour filtrer les fichiers
        use_bbox_annotation: Activer l'annotation des images
        use_document_annotation: Activer l'annotation du document
        sleep_seconds: Temps d'attente entre chaque document (pour free tier)
        
    Returns:
        Liste des résultats pour chaque document
    """
    input_path = Path(input_dir)
    files = list(input_path.glob(file_pattern))
    
    print(f"📂 {len(files)} fichiers trouvés dans '{input_dir}' avec le pattern '{file_pattern}'")
    print(f"⏱️ Pause de {sleep_seconds}s entre chaque document (free tier)")
    print(f"📁 Mode multi-pages: un markdown par page dans output_2/")
    print("=" * 60)
    
    results = []
    
    for i, file_path in enumerate(files, 1):
        print(f"\n[{i}/{len(files)}] Traitement de: {file_path.name}")
        print("-" * 60)
        
        try:
            result = run_ocr_with_annotations_per_page(
                file_path=str(file_path),
                use_bbox_annotation=use_bbox_annotation,
                use_document_annotation=use_document_annotation
            )
            
            if result:
                results.append({
                    "file": file_path.name,
                    "status": "success",
                    "output_dir": str(result["output_dir"]),
                    "pages_count": len(result["markdown_paths"]),
                    "images_count": len(result["images_info"])
                })
            else:
                results.append({
                    "file": file_path.name,
                    "status": "failed",
                    "error": "OCR failed"
                })
        except Exception as e:
            results.append({
                "file": file_path.name,
                "status": "error",
                "error": str(e)
            })
            print(f"❌ Erreur: {e}")
        
        # Pause entre chaque document pour respecter les limites du free tier
        if i < len(files):
            print(f"\n⏳ Pause de {sleep_seconds}s avant le prochain document...")
            time.sleep(sleep_seconds)
    
    # Résumé final
    print("\n" + "=" * 60)
    print("📊 RÉSUMÉ DU TRAITEMENT EN LOT (MODE MULTI-PAGES)")
    print("=" * 60)
    
    success_count = sum(1 for r in results if r["status"] == "success")
    failed_count = len(results) - success_count
    total_pages = sum(r.get("pages_count", 0) for r in results if r["status"] == "success")
    
    print(f"✅ Réussis: {success_count}/{len(results)}")
    print(f"❌ Échecs: {failed_count}/{len(results)}")
    print(f"📄 Total pages générées: {total_pages}")
    
    if success_count > 0:
        print(f"\n📁 Les fichiers ont été sauvegardés dans 'output_2/'")
        print("   Chaque sous-dossier contient:")
        print("   - Un fichier page_X.md par page avec frontmatter YAML")
        print("   - Un fichier _metadata.json avec les données globales")
        print("   - Un dossier images/ avec les images extraites")
    
    return results


print("✅ Fonction de traitement en lot multi-pages définie")

✅ Fonction de traitement en lot multi-pages définie


## Exécution : Traitement de tous les PDFs (mode multi-pages)

Lance le traitement de tous les fichiers PDF du dossier `ressources/` avec :
- Un markdown par page
- Pause de 2 secondes entre chaque document (free tier)
- Sortie dans `output_2/`

In [46]:
# Traiter TOUS les fichiers PDF du dossier ressources (MODE MULTI-PAGES)
# Chaque page aura son propre fichier markdown
# Pause de 2 secondes entre chaque document pour le free tier

batch_results_per_page = process_all_documents_per_page(
    input_dir="ressources",
    file_pattern="*.pdf",
    use_bbox_annotation=True,
    use_document_annotation=True,
    sleep_seconds=2.0  # Pause de 2 secondes pour free tier
)

📂 12 fichiers trouvés dans 'ressources' avec le pattern '*.pdf'
⏱️ Pause de 2.0s entre chaque document (free tier)
📁 Mode multi-pages: un markdown par page dans output_2/

[1/12] Traitement de: 23248.pdf
------------------------------------------------------------
🔄 Traitement du fichier (mode multi-pages): 23248.pdf
--------------------------------------------------
✅ Fichier encodé en base64
📄 Type de document: application/pdf
✅ Annotation BBox activée
✅ Annotation Document activée (max 8 pages)

🔄 Appel de l'API Mistral OCR...


✅ OCR terminé avec succès!

📁 Dossier de sortie créé: output_2\23248

🔄 Génération des fichiers markdown par page...
  ✅ Image sauvegardée: page1_img1.jpg
  ✅ Image sauvegardée: page5_img1.jpg
  ✅ Page 1/8 sauvegardée: page_1.md
  ✅ Page 2/8 sauvegardée: page_2.md
  ✅ Page 3/8 sauvegardée: page_3.md
  ✅ Page 4/8 sauvegardée: page_4.md
  ✅ Page 5/8 sauvegardée: page_5.md
  ✅ Page 6/8 sauvegardée: page_6.md
  ✅ Page 7/8 sauvegardée: page_7.md
  ✅ Page 8/8 sauvegardée: page_8.md
✅ Métadonnées sauvegardées: output_2\23248\23248_metadata.json

📊 Résumé:
   - Pages traitées: 8
   - Fichiers markdown créés: 8
   - Images extraites: 2
   - Dossier de sortie: output_2\23248

⏳ Pause de 2.0s avant le prochain document...

[2/12] Traitement de: 5-qst-1975-03-01.pdf
------------------------------------------------------------
🔄 Traitement du fichier (mode multi-pages): 5-qst-1975-03-01.pdf
--------------------------------------------------
✅ Fichier encodé en base64
📄 Type de document: application

In [47]:
# Afficher la structure du dossier de sortie
if result:
    output_dir = result["output_dir"]
    print(f"📁 Structure du dossier de sortie: {output_dir}")
    print("-" * 50)
    
    for item in output_dir.rglob("*"):
        if item.is_file():
            relative = item.relative_to(output_dir)
            size = item.stat().st_size
            print(f"   📄 {relative} ({size:,} bytes)")

📁 Structure du dossier de sortie: output\23248
--------------------------------------------------
   📄 23248.md (28,304 bytes)
   📄 23248_metadata.json (3,274 bytes)
   📄 images\page1_img1.jpg (4,176 bytes)
   📄 images\page5_img1.jpg (64,310 bytes)
